In [ ]:
import os
import h5py
import numpy as np

os.makedirs("../cadence_data", exist_ok=True)

def segment_file(file_path, chunk_sec=2.0):
    with h5py.File(file_path, "r") as f:
        time = f["time"][:]
        speed = f["speed"][:]
        vibration = f["vibration"][:]

        chunks = []
        start = 0
        while start < time[-1]:
            mask = (time >= start) & (time < start + chunk_sec)
            chunks.append((time[mask], speed[mask], vibration[mask]))
            start += chunk_sec

    # Save each chunk as a separate dataset in new HDF5
    out_path = f"../cadence_data/segmented_{os.path.basename(file_path)}"
    with h5py.File(out_path, "w") as f:
        for i, (t, s, v) in enumerate(chunks):
            grp = f.create_group(f"chunk_{i}")
            grp.create_dataset("time", data=t)
            grp.create_dataset("speed", data=s)
            grp.create_dataset("vibration", data=v)

files = os.listdir("../clean_data")
for file in files:
    segment_file(f"../clean_data/{file}")
    print(f"✂️ Cadence cut: {file}")
